# Maximum likelihood estimation via GMM

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import statsmodels.api as sm
import formulaic as fm

from gmm.gmm import GMMEstimator

import pdir

np.random.seed(94305)

## OLS

In [2]:
# simulate some data for for the linear probability model
n = 1000
p = 2
X = np.random.normal(size=(n, p))
X = sm.add_constant(X)
beta = np.array([0.5, -0.5, 0.5])
y = np.random.binomial(1, 1 / (1 + np.exp(-X @ beta)))

In [3]:
lpm_mod = sm.OLS(y, X)
lpm_res = lpm_mod.fit()
print("Parameters: ", lpm_res.params)

Parameters:  [ 0.60326474 -0.11124768  0.11141277]


In [5]:
Z = X.copy()

In [8]:
ψ = lambda z, y, x, beta: z * (y - x @ beta)[:, np.newaxis]
gmm = GMMEstimator(ψ, weighting_matrix=None, backend = "scipy")
gmm.fit(Z, y, X)
gmm.summary()

,coef,std err,t,p-value,[0.025,0.975]
0,0.6033,0.0317,19.0085,0.0000,0.5411,0.6655
1,-0.1112,0.0348,-3.1953,0.0014,-0.1795,-0.0430
2,0.1114,0.0310,3.5930,0.0003,0.0506,0.1722


## Logit

In [9]:
logit_mod = sm.Logit(y, X)
logit_res = logit_mod.fit(disp=0)
print("Parameters: ", logit_res.params)

Parameters:  [ 0.47180703 -0.53023967  0.5246359 ]


Moment condition:

$$
\mathbb{E} \left[ \left( y_i - \frac{1}{1 + \exp(-x_i' \beta)} \right) x_i \right] = 0
$$

In [10]:
ψ_logit = lambda z, y, x, beta: z * (y - (1 / (1 + np.exp(-x @ beta))))[:, np.newaxis]

gmm = GMMEstimator(ψ_logit)
gmm.fit(Z, y, X)
gmm.summary()


,coef,std err,t,p-value,[0.025,0.975]
0,0.4718,0.0147,32.1376,0.0,0.4430,0.5006
1,-0.5302,0.0142,-37.2980,0.0,-0.5581,-0.5024
2,0.5246,0.0140,37.5619,0.0,0.4973,0.5520


Identical

## Probit

In [13]:
from scipy.stats import norm
# simulate some data for probit
n, p = 1000, 2
X = np.random.normal(size=(n, p))
X = sm.add_constant(X)
beta = np.array([0.5, -0.5, 0.5])
y = np.random.binomial(1, norm.cdf(X @ beta))

In [14]:
probit_mod = sm.Probit(y, X)
probit_res = probit_mod.fit(disp=0)
print("Parameters: ", probit_res.params)

Parameters:  [ 0.47693552 -0.50799746  0.49620263]


Moment condition
$$
\mathbb{E} \left[ \left( y_i - \Phi(x_i' \beta) \right) x_i \right] = 0
$$

## Poisson

In [16]:
Z = X.copy()
ψ_probit = lambda z, y, x, beta: z * (y - norm.cdf(x @ beta))[:, np.newaxis]
gmm = GMMEstimator(ψ_probit)
gmm.fit(Z, y, X)
gmm.summary()

,coef,std err,t,p-value,[0.025,0.975]
0,0.4817,0.0134,35.8473,0.0,0.4553,0.5080
1,-0.5123,0.0121,-42.3092,0.0,-0.5361,-0.4886
2,0.4999,0.0122,40.9900,0.0,0.4760,0.5238


In [19]:
# poisson dgp
n, p = 1000, 2
X = np.random.normal(size=(n, p))
X = sm.add_constant(X)
beta = np.array([0.5, -0.5, 0.5])
y = np.random.poisson(np.exp(X @ beta))

Z = X.copy()

In [18]:
poisson_mod = sm.GLM(y, X, family=sm.families.Poisson())
poisson_res = poisson_mod.fit()
print("Parameters: ", poisson_res.params)

Parameters:  [ 0.49665406 -0.5116213   0.49354614]


In [21]:
ψ_poisson = lambda z, y, x, beta: z * (y - np.exp(x @ beta))[:, np.newaxis]
m = GMMEstimator(ψ_poisson, weighting_matrix=None, backend = "scipy")
m.fit(Z, y, X)
m.summary()

,coef,std err,t,p-value,[0.025,0.975]
0,0.5079,0.0317,16.0334,0.0,0.4458,0.5700
1,-0.5164,0.0316,-16.3216,0.0,-0.5784,-0.4544
2,0.4803,0.0307,15.6275,0.0,0.4201,0.5406


# General Maximum Likelihood 
Moment condition is the score of the likelihood

$$
g(\beta) = \sum_i \frac{\partial \log L(y_i, x_i, \beta)}{\partial \beta}
$$

## example of mixture logit

Let us write an example of a maximum likelihood model for a mixture model with two logit regressions with separate coefficient vectors. Each unit has probability $\lambda$ of being in the first regime and $1 - \lambda$ of being in the second regime. The likelihood is

$$
L(y, x, \beta_1, \beta_2, \lambda) = \prod_i \left( \lambda \frac{1}{1 + \exp(-x_i' \beta_1)} + (1 - \lambda) \frac{1}{1 + \exp(-x_i' \beta_2)} \right)^{y_i} \left( 1 - \lambda \frac{1}{1 + \exp(-x_i' \beta_1)} - (1 - \lambda) \frac{1}{1 + \exp(-x_i' \beta_2)} \right)^{1 - y_i}
$$

The log likelihood is 

$$
\log L(y, x, \beta_1, \beta_2, \lambda) = \sum_i y_i \log \left( \lambda \frac{1}{1 + \exp(-x_i' \beta_1)} + (1 - \lambda) \frac{1}{1 + \exp(-x_i' \beta_2)} \right) + (1 - y_i) \log \left( 1 - \lambda \frac{1}{1 + \exp(-x_i' \beta_1)} - (1 - \lambda) \frac{1}{1 + \exp(-x_i' \beta_2)} \right)
$$

The score is

$$
g(\beta_1, \beta_2, \lambda) = \sum_i \frac{\partial \log L(y_i, x_i, \beta_1, \beta_2, \lambda)}{\partial \beta_1} + \frac{\partial \log L(y_i, x_i, \beta_1, \beta_2, \lambda)}{\partial \beta_2} + \frac{\partial \log L(y_i, x_i, \beta_1, \beta_2, \lambda)}{\partial \lambda}
$$

In [29]:
import numpy as np
import scipy.stats as stats
import scipy.optimize as optimize

def logit(x, beta):
    return 1 / (1 + np.exp(-x @ beta))

n, p = 1000, 2
X = np.random.normal(size=(n, p))
X = sm.add_constant(X)
beta1 = np.array([0.5, -0.5, 0.5])
beta2 = np.array([-1, 0, -1])
lambda_ = 0.5
y = np.random.binomial(1, lambda_ * logit(X, beta1) + (1 - lambda_) * logit(X, beta2))
Z = X.copy()